# Monitoramento de Entrada e Saída

In [1]:
import requests
import numpy as np
import cv2
from deepface import DeepFace
import face_recognition
from io import BytesIO
from PIL import Image

In [2]:
# inicializar a url de acesso a api
base_url = 'http://localhost:3333'

In [4]:
# listar os usuários cadastrados
response = requests.get(f'{base_url}/user/list')
users = response.json()
users

[{'id': 'd3e3b8ea-788d-45d5-bca3-4027c2728715',
  'name': 'Laciene Garcia',
  'avatar_url': 'http://localhost:3333/uploads/54ac0182d235-laciene2.jpeg'},
 {'id': '4ba22232-b5aa-474a-91a6-6898abd3ab27',
  'name': 'Leonardo Gonçalves',
  'avatar_url': 'http://localhost:3333/uploads/1c9ce47f65b0-perfil-leo.jpeg'}]

In [6]:
known_faces = []

# preparar lista de objetos
for user in users:
    image_link = user['avatar_url']

    # baixar image
    fetch_image = requests.get(image_link)
    binary_encoding = BytesIO(fetch_image.content)

    # converter para o formato aceito pelo face-recognition
    convert_to_RGB = Image.open(binary_encoding).convert("RGB")
    convert_to_array = np.array(convert_to_RGB)

    # codificar imagem
    facial_encoding = face_recognition.face_encodings(convert_to_array)[0]

    # adicionar a lista de objetos
    known_faces.append({ 'id': user['id'], 'face_encoding': facial_encoding, 'name': user['name']})

In [7]:
# iniciar a webcam
video_capture = cv2.VideoCapture(0)

In [ ]:
while True: 
    # capturando um frame
    ret, frame = video_capture.read()

    # encontra face no frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # demarcar o rosto com um quadrado
        cv2.rectangle(img=frame, pt1=(left, top), pt2=(right, bottom), color=(0, 255, 0), thickness=2)

        # verifica se as faces correspondem
        in_matched = face_recognition.compare_faces([user['face_encoding'] for user in known_faces], face_encoding)

        name, user_id = "Desconhecido", None
        if(True in in_matched):
            index = in_matched.index(True)
            user_id = known_faces[index]['id']
            name = known_faces[index]['name']

            # região de interesse
            roi = frame[max(0, top):min(frame.shape[0], bottom), max(0, left):min(frame.shape[1], right)]
            emotion = DeepFace.analyze(roi, actions="emotion", enforce_detection=False)[0]["dominant_emotion"]

            # registrar presença
            requests.post(f'{base_url}/presence/create', 
                          json={"user_id": user_id, "mood": emotion}
                          )
            
            # adicionar as identificações do frame
            font, scale, color, thickness = cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1

            # adicionar as identificações no frame
            cv2.putText(img=frame, text=f'id: {user_id}', org=(left + 6, bottom + 20), fontFace=font, fontScale=scale, color=color, thickness=thickness)
            cv2.putText(img=frame, text=f'name: {name}', org=(left + 6, bottom + 40), fontFace=font, fontScale=scale, color=color, thickness=thickness)
            cv2.putText(img=frame, text=f'emotion: {emotion}', org=(left + 6, bottom + 60), fontFace=font, fontScale=scale, color=color, thickness=thickness)

    # mostrar o frame
    cv2.imshow('Video', frame)

    # sair do loop
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break


# liberar recursos
video_capture.release()
cv2.destroyAllWindows()

Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 29.02it/s]
2025-01-10 20:50:33.260 Python[19401:50184776] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-10 20:50:33.260 Python[19401:50184776] +[IMKInputSession subclass]: chose IMKInputSession_Modern
Action: emotion:   0%|          | 0/1 [00:00<?, ?it/s]